# 3D reconstruction using 2 image

Key References
- https://www.opencvhelp.org/tutorials/advanced/reconstruction-opencv/

In [ ]:
from __future__ import annotations

from typing import TYPE_CHECKING

import cv2 as cv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial.transform as sci_trans

import project_3d_reconstruction.mpl as proj_mpl

if TYPE_CHECKING:
    from mpl_toolkits.mplot3d.axes3d import Axes3D

# Generate simple scene

In [ ]:
x, y, z = np.indices((16, 16, 16))

cube1 = (x < 6) & (y < 6) & (z < 6)  # noqa: PLR2004
cube2 = (x >= 10) & (y >= 10) & (z >= 10)  # noqa: PLR2004
link = abs(x - y) + abs(y - z) + abs(z - x) <= 6  # noqa: PLR2004

voxelarray = cube1 | cube2 | link

colors = np.empty((*voxelarray.shape, 3))
colors[link] = mpl.colors.to_rgb("tab:red")
colors[cube1] = mpl.colors.to_rgb("tab:blue")
colors[cube2] = mpl.colors.to_rgb("tab:green")

rng = np.random.default_rng()
colors = np.clip(colors + rng.uniform(-0.5, 0.5, size=colors.shape), 0, 1)

ax: Axes3D
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8, 8))
ax.axis("off")
ax.voxels(voxelarray, facecolors=colors, edgecolors=colors)
fig.tight_layout()

# Save image from two perspectives.

In [ ]:
ax.view_init(elev=30, azim=-60)
image_1 = proj_mpl.fig_export_rgba(fig)
eye_1 = proj_mpl.axes_eye_coordinate(ax)
dir_1 = proj_mpl.axes_eye_direction(ax)

ax.view_init(elev=20, azim=-50)
image_2 = proj_mpl.fig_export_rgba(fig)
eye_2 = proj_mpl.axes_eye_coordinate(ax)
dir_2 = proj_mpl.axes_eye_direction(ax)

gray_1 = cv.cvtColor(image_1, cv.COLOR_BGR2GRAY)
gray_2 = cv.cvtColor(image_2, cv.COLOR_BGR2GRAY)

ifig, iaxs = plt.subplots(ncols=2, figsize=(8, 8))
iaxs[0].imshow(image_1)
iaxs[1].imshow(image_2)
ifig.tight_layout()

# Detect SIFT features

In [ ]:
sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray_1, None)
kp2, des2 = sift.detectAndCompute(gray_2, None)

image_kp_1 = cv.drawKeypoints(
    gray_1, kp1, image_1, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
image_kp_2 = cv.drawKeypoints(
    gray_2, kp2, image_2, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

ifig, iaxs = plt.subplots(ncols=2, figsize=(8, 8))
iaxs[0].imshow(image_kp_1)
iaxs[1].imshow(image_kp_2);

# Match features (brute-force)

In [ ]:
bf = cv.BFMatcher()
bf_matches = bf.knnMatch(des1, des2, k=2)

good = []
for m, n in bf_matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])

image_matches = cv.drawMatchesKnn(
    image_1,
    kp1,
    image_2,
    kp2,
    good,
    None,
    flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)


ifig, iax = plt.subplots()
iax.imshow(image_matches)
ifig.tight_layout()

# Estimate essential matrix

In [ ]:
points_1 = np.array([kp1[match.queryIdx].pt for [match] in good])[:, np.newaxis, :]
points_2 = np.array([kp2[match.trainIdx].pt for [match] in good])[:, np.newaxis, :]

intrinsic_mat = np.eye(3)
essential_mat, mask = cv.findEssentialMat(
    points_1, points_2, intrinsic_mat, method=cv.RANSAC, prob=1 - 1e-4, threshold=1
)

_, est_rot, est_trans, _ = cv.recoverPose(
    essential_mat, points_1, points_2, intrinsic_mat, mask=mask
)

In [ ]:
est_extrinsic = np.hstack((est_rot, est_trans))

act_rot = sci_trans.Rotation.align_vectors(dir_2, dir_1)[0].as_matrix()
act_extrinsic = np.hstack((act_rot, np.reshape(dir_2 - dir_1, (-1, 1))))
origin = np.hstack((np.eye(3), np.zeros((3, 1))))

all_extrinsics = [origin, act_extrinsic, est_extrinsic]
base_colors = [
    mpl.colors.to_rgb("tab:blue"),
    mpl.colors.to_rgb("tab:green"),
    mpl.colors.to_rgb("tab:red"),
]
cfig, cax = plt.subplots(subplot_kw={"projection": "3d"})
for mat, color in zip(all_extrinsics, base_colors, strict=False):
    pos = mat[:, 3]
    for column, weight in zip(mat[:, :3].T, [1, 0.8, 0.5], strict=False):
        cax.quiver(*pos, *column, color=np.array(color) * weight)

fig.tight_layout()
cax.set_xlim3d(-1, 1)
cax.set_ylim3d(-1, 1)
cax.set_zlim3d(-1, 1)
# cax.voxels(voxelarray, facecolors=colors, edgecolors=colors);

(-1.0, 1.0)